In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso\Bruit')

import pandas as pd
import altair as alt
import datetime as dti
import locale
from Emission import

locale.setlocale(locale.LC_ALL, 'fr_FR')

# ExD936  
sur cette voie il va falloir extrapoler des données : 
- sur la première semaines de mesure
- sur un des deux sens pendant la deuxième semaine de mesure

> ## comparaison des données Cerema et BdxMet
il s'agit de comparer les données Bdx Met avec les données Cerema, pour pouvoir extrapoler les valeurs sur la 2ème semaine de mesures acoustiques dans les deux sens

In [3]:
# paramètres 
fichierCerema = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\du_29-03-2022_au_04-04-2022_Cerema\Mesures_individuelles.xlsx'
fichierBM = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\du_04-04-2022 au 18-04-2022_CEPV\CSV\Semaine1\Vers_Bordeaux\00030093-20220331123748.CSV'

In [4]:
# import des données et mise en forme
dfCerema = pd.read_excel(fichierCerema, names=['horodatage', 'voie', 'contresens', 'vitesse', 'longueur', 'classe_veh'])
dfBm = pd.read_csv(fichierBM, names=['date', 'heure', 'vitesse', 'longueur', 'div', 'voie', 'reverse'] , skiprows=5)
dfBm['horodatage'] = pd.to_datetime(dfBm.date + ' ' + dfBm.heure, dayfirst=True)
dfCerema['classe_veh'] = dfCerema.longueur.apply(lambda x: 'vl' if x <= 6 else 'pl')
dfBm['classe_veh'] = dfBm.longueur.apply(lambda x: 'vl' if x <= 6 else 'pl')

In [95]:
# Comparaison des données
dfBm30min = dfBm.groupby([pd.Grouper(freq='30min', key='horodatage'), 'classe_veh']).date.count().reset_index().rename(
    columns={'date': 'nb_veh'}).sort_values(['horodatage', 'classe_veh'])
dfCerema30min = dfCerema.groupby([pd.Grouper(freq='30min', key='horodatage'), 'classe_veh']).voie.count().reset_index().rename(
    columns={'voie': 'nb_veh'}).sort_values(['horodatage', 'classe_veh'])
jointureBmCerema30min = dfBm30min.merge(dfCerema30min, on=['horodatage', 'classe_veh'], suffixes=['_bm', '_cerema'])[2:-2]
dfDiffPc = jointureBmCerema30min.groupby('horodatage').apply(
        lambda x: 1 - (sum(x.nb_veh_bm) / sum(x.nb_veh_cerema))).reset_index().rename(columns={0: 'pc_diff_bm_cerema'})

In [163]:
# Visualisation
dFGraph = pd.melt(jointureBmCerema30min.rename(columns={'nb_veh_bm': 'Bordeaux Metropole', 'nb_veh_cerema': 'Cerema'}),
                  id_vars=(['horodatage', 'classe_veh']), value_vars=('Bordeaux Metropole', 'Cerema'),
                  var_name='src_mesure')[1:-1]
listChart, listChartMirroir = [], []
largeurChart = 500
formatDate = '%A %d %B'
for dateTest in jointureBmCerema30min.horodatage.dt.strftime('%Y-%m-%d').unique():
    # graph de comparaison TV
    titre = f'Véhicules selon source ; le {dti.datetime.fromisoformat(dateTest).strftime(formatDate)}'
    graphNbVeh = alt.Chart(dFGraph.loc[(dFGraph.src_mesure.isin(('Bordeaux Metropole', 'Cerema'))) &
                                       (dFGraph.horodatage.dt.date == pd.to_datetime(dateTest))], title=titre, width=largeurChart).mark_line().encode(
        x=alt.X('hoursminutes(horodatage):O', title='heure'),
        y=alt.Y('sum(value):Q', title='Nombre de véhicules'),
        color=alt.Color('src_mesure:N', scale=alt.Scale(scheme='dark2'), legend=alt.Legend(values=['Bordeaux Metropole', 'Cerema'], title="Source des mesures"))
    )
    graphDiffNbVeh = alt.Chart(dfDiffPc.loc[dfDiffPc.horodatage.dt.date == pd.to_datetime(dateTest)]).mark_bar(opacity=0.3, color='gray').encode(
        x=alt.X('hoursminutes(horodatage):O', title='heure'),
        y=alt.Y('pc_diff_bm_cerema:Q', title='Pourcentage de variation', axis=alt.Axis(titleColor='gray')))
    listChart.append(alt.layer(graphNbVeh, graphDiffNbVeh).resolve_scale(y='independent'))

    # graph de comparaison VL / PL
    titreMirroir = f'Type de véhicules selon source ; le {dti.datetime.fromisoformat(dateTest).strftime(formatDate)}'
    graphMirroir = alt.Chart(dFGraph.loc[dFGraph.horodatage.dt.date == pd.to_datetime(dateTest)].assign(
        type_veh=dFGraph.src_mesure+'_'+dFGraph.classe_veh, value=dFGraph.apply(
            lambda x: 0-x.value if x.src_mesure == 'Cerema' else x.value, axis=1)), title=titreMirroir, width=largeurChart).mark_bar(
    ).encode(
    x=alt.X('hoursminutes(horodatage):O', title='Heure'),
    y=alt.Y('value:Q', title='Nombre de véhicules', axis=alt.Axis(labelExpr='abs(datum.value)')),
    color=alt.Color('type_veh', title='Type de véhicule', legend=alt.Legend(values=['Bordeaux Metropole_pl','Bordeaux Metropole_vl', 'Cerema_pl', 'Cerema_vl'])))
    listChartMirroir.append(graphMirroir)

graphCompCeremaBdxMet = alt.vconcat(*listChart)
graphMirroirComplet = alt.vconcat(*listChartMirroir)
graphCompTotalTrafic = (graphCompCeremaBdxMet | graphMirroirComplet).resolve_legend('independent')

<div class="alert alert-block alert-warning"><b>Résultats:</b> Il y a une forte divergence des mesures en tout véhicules entre les données fournies par Bordeaux Métropole et celle mesurées par le Cerea, sur la période de recouvrement. Malheureusement cette période ne couvre que du vendredi 01 au lundi 04, mais les résultats sur les heures de pointes indique des variations pouvant aller jusqu'à 50%</div>

In [165]:
jointureBmCerema30min

,horodatage,classe_veh,nb_veh_bm,nb_veh_cerema
2,2022-03-31 13:00:00,pl,36,49
3,2022-03-31 13:00:00,vl,353,422
4,2022-03-31 13:30:00,pl,31,45
5,2022-03-31 13:30:00,vl,412,517
6,2022-03-31 14:00:00,pl,35,36
...,...,...,...,...
349,2022-04-04 09:00:00,vl,446,557
350,2022-04-04 09:30:00,pl,48,54
351,2022-04-04 09:30:00,vl,407,501
352,2022-04-04 10:00:00,pl,42,51
